In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import nnfs
from nnfs.datasets import sine_data, spiral_data
import random
import requests
from NNS import NeuralNetwork as NN #import neural net code from github to reduce copy/pasting
from NNS import load_MNIST_Data
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
import os

# Saving and Loading Models and Their Paraneters

## Retrieving Parameters

In [ ]:
# Add this to Dense layer
def get_parameters(self):
    return self.weights, self.biases

# Add this to Model class
# Retrieves and returns parameters of trainable layers
def get_parameters (self):
    
    # Create a list for parameters
    parameters = []
    
    # Iterable trainable layers and get their parameters
    for layer in self.trainable_layers:
        parameters.append(layer.get_parameters())
        
    # Return a list
    return parameters

## Setting Parameters

In [1]:
# Add this to Dense Layer

# Set weights and biases in a layer instance
def set_parameters(self, weights, biases):
    self.weights = weights
    self.biases = biases
    
    
# Add this to the Model Class
  
# Updates the model with new parameters
def set_parameters(self, parameters):
    # Iterate over the parameters and layers
    # and update each layers with each set of the parameters
    for parameter_set, layer in zip(parameters, self.trainable_layers):
        layer.set_parameters( * parameter_set)
        
# Update Model Finalize function to account for not needing an optimizer
def finalize(self):
    """"""
    # Update loss object with trainable layers
    if self.loss is not None:
        self.loss.remember_trainable_layers(self.trainable_layers)
        
# Update Model Set function to account for taking in paramters
# Set loss, optimizer and accuracy
def set(self , *, loss = None, optimizer = None, accuracy = None):
    
    if loss is not None:
        self.loss = loss
    
    if optimizer is not None :
        self.optimizer = optimizer
    
    if accuracy is not None :
        self.accuracy = accuracy

## Saving Parameters

In [ ]:
import pickle # For serializing  nidek ivject

# Add save parameters function to model
# Saves the parameters to a file
def save_parameters(self, path):
    
    # Open a file in the binary-write mode and save parameters to it
    with open (path, 'wb') as f:
        pickle.dump(self.get_parameters(), f)

## Loading Parameters

In [ ]:
# Loads the weights and updates a model instance with them
def load_parameters(self, path):
    # Open file in the binary-read mode,
    # load weights and update trainable layers
    with open (path, 'rb' ) as f:
        self.set_parameters(pickle.load(f))

## Saving the Model

In [ ]:
# Saves the model
import copy

def save(self, path):
    
    # Make a deep copy of current model instance
    model = copy.deepcopy(self)
    
    # Reset accumulated values in loss and accuracy objects
    model.loss.new_pass()
    model.accuracy.new_pass()
    
    # Remove data from the input layer and gradients from the loss object
    model.input_layer.__dict__.pop('output' , None)
    model.loss.__dict__.pop('dinputs', None)
    
    # For each layer remove inputs, output and dinputs properties
    for layer in model.layers:
        for property in [ 'inputs', 'output', 'dinputs', 'dweights', 'dbiases']:
            layer.__dict__.pop( property , None )
    
    # Open a file in the binary-write mode and save the model
    with open (path, 'wb' ) as f:
        pickle.dump(model, f)

## Loading the model

In [ ]:
# Loads and returns a model
@staticmethod #create a model object without first needing to instantiate a model object
def load(path):
    
    # Open file in the binary-read mode, load a model
    with open (path, 'rb' ) as f:
        model = pickle.load(f)
        
    # Return a model
    return model